In [1]:
import pandas as pd
import numpy  as np
import os
import os.path
import shutil
import sys
from gc import collect as gc_collect


# initialize

In [2]:
%load_ext autoreload
%autoreload 2

from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

scaling_cadence = 3
scale_cadence = scaling_cadence
run_list = [
#                 'msis2',
#                 'jb2008'
                'dtm2020_o',
            ]


# To Do:
Exat naming conventions
Check the external att files 

In [3]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os
from datetime import datetime,timedelta

obj = {}

# file_raw_ICs = f"{g2b_path}Spire083_RawEphem_20181101_20181130.txt"
# file_g2b     = f"pce_spire083_leoOrb_20181101_20181130"



for i,den in enumerate(run_list):
    settings_ice2= {# Basic input settings
                 'satellite'            : {'input': 'icesat2'},
                 'den_model'            : {'input': den},
                 'initialize_satellite' : {'input': True},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'epoch_start'    : {'input': ["2019-04-28",
                                               "2019-06-03" ]},


#                  'run_specifier'  : {'input': '_'},
#                  'cd_model'       : {'input': 'DRIA'},
#                  'file_string'    : {'input': 'DRIAscaled_'},
#                  'model_data_path' : {'input': run_dict[den]['model_path']},
#                  'verbose' : {'input': True},
#                  # Force Model settings
#                   'cd_value'              : {'input':1.0 },
#                   'scaling_factor'        : {'input':True},
#                   'hours_between_cd_adj'  : {'input':scaling_cadence},
#                   #### Comment for unadjusted run:
#                   'cd_adjustment_boolean' : {'input':True },
#                 #### DRIA CD Model Parameters
#                 'cd_model_params' : {'input':{ 
#                         'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
#                         'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
#                         'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient, Alpha is b/w 0 and 1
#                         'KL'     : '0.0D0'    ,    #!  langmuir parameter
#                         'FRACOX' : '1.0D0'   ,     #!  fraction of surface covered by atomic oxygen
#                    }},
#                   #### ---------------------------------------
#                  # Run


# #             scaleparameter_times
#                   'arc_type'       : {'input':'Nominal30hr'},      
#                   'step'           : {'input': 60.},
#                   'orbfil_step'    : {'input': 120.},
# #                   'which_ICfile'   : {'input':file_raw_ICs},
# #                   'which_g2bfile'  : {'input':file_g2b},
#                     #
# #                   'arc'            : {'input': arc_start},
# #                   'epoch_start'    : {'input': epoch_startDT},
# #                   'epoch_stop'     : {'input': epoch_endDT},  
# #                 'scaleparameter_times' : {'input': scaleparameter_times},  
#                    #                                
# #                   'global_options' : {'input':'pso_2018'},
#                  # Request read on raw outputs
#                   'request_data'   : {'input': ['Trajectory_orbfil', 
#                                                'Density', 
#                                                'Residuals_summary',
#                                                'DragFile',
#                                                'AdjustedParams'
#                                                ]},
              #end dict
     }

startdate = "2019-03-28"  
enddate   = "2019-05-03"  

sat = Pygeodyn(settings_ice2, use_file=False)
sat.initialize_timeperiod_stage1(startdate, enddate,
                             overwrite_exat=False, 
                             overwrite_ICtext=True)
sat = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using the ICESat-2 Class
['2019-04-28', '2019-06-03']
  * Global Option not needed during Initializing Stage
check sat ICESat2
----------------------------------------------------------------------------
Initializing the time period from 2019-03-28 00:00:00 to 2019-05-04 00:00:00
     overwriting the epoch start and stop to match
     self.prms['arc'] ['2019.090', '2019.091A', '2019.092A', '2019.093', '2019.094', '2019.095', '2019.096', '2019.097', '2019.098', '2019.099', '2019.100', '2019.101', '2019.102', '2019.103', '2019.104', '2019.105', '2019.106A', '2019.107A', '2019.108', '2019.109', '2019.110', '2019.111', '2019.112', '2019.113', '2019.114', '2019.115', '2019.116', '2019.117', '2019.118', '2019.119', '2019.120', '2019.121', '2019.122A', '2019.123A']
----------------------------------------------------------------------------

Using the ICESat-2 Class
[Timestamp('2019-03-31 00:00:00'), Times

rm: cannot remove ‘/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/pce_icesat2_pso_20190331_20190503’: No such file or directory


        - pce_fortran.f executed


In [4]:
import sys
sys.exit()

SystemExit: 

/data/miniconda3/envs/pygeodyn2/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Process Raw Ephemerides-Test

## read binary orbfil

In [ ]:
from pygeodyn.util_dir.time_systems import time_mjdsecs_tdt_to_utc
from pygeodyn.util_dir.time_systems import mjds_to_ymdhms

def read_binary_ORBFIL(orb_fil):

    from scipy.io import FortranFile
    # orb_fil = file
    f = FortranFile(orb_fil, 'r')

    #### -----------------------------------------------------
    #### ------------------- HEADER RECORD -------------------
    #### -----------------------------------------------------
    ### Read the first record, this is the header buffer
    a = f.read_record(float)  # read the record with the required datatype


    #### Glean important variables
    NA     = int(a[2-1]) # Number of alphanumeric buffers to follow the header
    NC     = int(a[3-1]) # Number of card images in the GEODYN II input control deck
    NSATS  = int(a[7-1])  # Number of satellites on this file:  
    NWDSAT = int(a[8-1])  # Actual number of words per satellite per time point (NWDSAT <= 39).
    NWDATA = int(a[9-1])   #NSATS*NWDSAT
    NTIMBF = int(a[10-1]) # Number of time points per Data Buffer

    header= {}
    header['Number of alphanumeric data buffers to follow (NA)']             = a[2-1]
    header['Number of card images in the GEODYN II input control deck (NC)'] = a[3-1]
    header['Arc Number.']                                                    = a[4-1]
    header['Global Iteration Number']                                        = a[5-1]
    header['Inner Iteration Number']                                         = a[6-1]
    header['Number of satellites on this file']                              = a[7-1]  # upper limit of 50
    header['Actual number of words per satellite per time point']            = a[8-1]
    header['Number of words of data per time point (NWDATA=NSATS*NWDSAT)']   = a[9-1]
    header['Number of time points per Data Buffer (NTIMBF)']                 = a[10-1]
    header['Trajectory Start Date & Time in form YYMMDDHHMMSS .0D0 UTC']     = a[11-1]
    header['Fractional seconds of Start Time. UTC']                          = a[12-1]
    header['Trajectory Stop Date & Time in form YYMMDDHHMMSS .0D0 UTC']      = a[13-1]
    header['Fractional seconds of Stop Time. UTC']                           = a[14-1]
    header['Trajectory Start Date & Time in MJDS']                           = a[15-1] # (MJDS=(JD -2430000.5 D0 )*86400+ ISEC) ET
    header['Fractional seconds of Start Time']                               = a[16-1]
    header['Trajectory Stop Date & Time in MJDS']                            = a[17-1] # (MJDS=(JD -2430000.5 D0 )*86400+ ISEC) ET
    header['Fractional seconds of Stop Time. ET']                            = a[18-1]
    header['Nominal interval between trajectory times in seconds. ET ']      = a[19-1]
    header['Nominal number of trajectory times.']                            = a[20-1]
    header['Output S/C ephem ref sys(0 = TOD, 1 = TOR, 2 =  J2000)']         = a[22-1]
    # 
    header['Speed of Light.']                                     = a[101-1]
    header['GM for Earth.']                                       = a[102-1]
    header['Semi -major axis of Earth reference ellipsoid.']      = a[103-1]
    header['Equatorial Flattening of Earth reference ellipsoid.'] = a[104-1]
    header['Gravitational Potential Checksum.']                   = a[105-1]
    header['Maximum Degree of Gravitational Expansion.']          = a[106-1]
    header['Maximum Order of Gravitational Expansion.']           = a[107-1]   ### SKIP from 108 -200
    #
    #### PRESENCE ON FILE INDICATORS
    ## right ascension of Greenwich 
    header['Presence of right ascension of Greenwich for each time point in each Buffer'] = a[201-1]   
    ## Inertial State Vector
    header['Presence per Sat. of inertial X coordinate for each time point']    = a[202-1]   
    header['Presence per Sat. of inertial Y coordinate for each time point']    = a[203-1]   
    header['Presence per Sat. of inertial Z coordinate for each time point']    = a[204-1]   
    header['Presence per Sat. of inertial Xdot coordinate for each time point'] = a[205-1]   
    header['Presence per Sat. of inertial Ydot coordinate for each time point'] = a[206-1]   
    header['Presence per Sat. of inertial Zdot coordinate for each time point'] = a[207-1] 
    # 
    header['Presence per Sat. of geodetic latitude for each time point'] = a[208-1]   
    header['Presence per Sat. of east longitude for each time point']    = a[209-1]   
    # 
    header['Presence per Sat. of ECF X coordinate for each time point']  = a[210-1]   
    header['Presence per Sat. of ECF Y coordinate for each time point']  = a[211-1]   
    header['Presence per Sat. of ECF Z coordinate for each time point']  = a[212-1]   
    header['Presence per Sat. of ECF Xdot for each time point']         = a[213-1]   
    header['Presence per Sat. of ECF Ydot for each time point']         = a[214-1]   
    header['Presence per Sat. of ECF Zdot for each time point']         = a[215-1]   
    header['Presence per Sat. of polar motion X for each time point']   = a[216-1]   
    header['Presence per Sat. of polar motion Y for each time point']   = a[217-1]   
    header['Presence per Sat. of beta prime angle for each time point'] = a[218-1]   
    header['Presence per Sat. of yaw angle for each time point']        = a[219-1]   
    header['Presence per Sat. of orbit angle for each time point']      = a[220-1]   



    # break
    ##### Satellite ID ’s for all Satellites on File.
    ###       Trajectory data is ordered based upon order of these Satellite ID ’s.'
    for i in range(int(NSATS)):
        ii = i + 1
        index_sats = 300 + (ii)
        header['Satellite '+str(ii)+' ID'] = a[index_sats-1]  


    #### ----------------------------------------------------
    #### --------------- ALPHANUMERIC RECORDS ---------------
    #### ----------------------------------------------------
    #### We don't care about the Alphanumeric buffers so skip over them.
    for i in range(int(NA)):
        a = f.read_record(float)


    #### -----------------------------------------------------
    #### -------------- DATA + SENTINEL RECORDS --------------
    #### -----------------------------------------------------
    ### Read the Data records in a while loop.  
    ### When we hit the end_data_val, we have reached the
    ###    sentinel record and we can exit the while loop 
    ###    to read in the sentinel buffer.


    end_data_val           = 9000000000
    end_datarecord         = False
    data_dict_times        = {}
    data_dict_RA_greenwich = {}
    data_dict_sat_packets  = {}

    count_while = 0

    data_dict_sat_packets['MJDSEC ET']                       =[]
    data_dict_sat_packets['X j2000'] =[]
    data_dict_sat_packets['Y j2000'] =[]
    data_dict_sat_packets['Z j2000'] =[]
    data_dict_sat_packets['X_dot j2000']   =[]
    data_dict_sat_packets['Y_dot j2000']   =[]
    data_dict_sat_packets['Z_dot j2000']   =[]
    data_dict_sat_packets['Geodetic Latitude']     =[]
    data_dict_sat_packets['East Longitude']        =[]
    data_dict_sat_packets['Height']                =[]
    #         data_dict_sat_packets['Satellite ECF X coordinate']      =[]
    #         data_dict_sat_packets['Satellite ECF Y coordinate']      =[]
    #         data_dict_sat_packets['Satellite ECF Z coordinate']      =[]
    #         data_dict_sat_packets['Satellite ECF X velocity']        =[]
    #         data_dict_sat_packets['Satellite ECF Y velocity']        =[]
    #         data_dict_sat_packets['Satellite ECF Z velocity']        =[]
    #         data_dict_sat_packets['Polar Motion X']                  =[]
    #         data_dict_sat_packets['Polar Motion Y']                  =[]
    #         data_dict_sat_packets['Beta prime angle']                =[]
    #         data_dict_sat_packets['Yaw angle']                       =[]
    #         data_dict_sat_packets['Orbit Angle']                     =[]
    #         data_dict_sat_packets['Q(1)']                            =[]
    #         data_dict_sat_packets['Q(2)']                            =[]
    #         data_dict_sat_packets['Q(3)']                            =[]
    #         data_dict_sat_packets['Q(4)']                            =[]

    while end_datarecord == False:
        ### Read in each data buffer
        a = f.read_record(float)

        if not end_data_val in a:
            count_while+=1
            NTB    = int(a[5-1])  # Number of trajectory times in this Data Buffer (NTB <= NTIMBF ).
            MJDSBF = a[4-1]

            #### Trajectory Times in elapsed ET seconds from MJDSBF
            counter = 0
            for itime in np.arange( (6)   ,   ((NTB+5)  +1)  ):
                index_times = int(itime)
                data_dict_times[counter] = str(MJDSBF + a[index_times-1] )
                counter+=1

    #             if counter <= 100:
    #                 print(MJDSBF + a[index_times-1])


            #### Right Ascension of Greenwich Values (radians) for each time in Buffer.
            counter = 0
            for i in np.arange((NTIMBF+6) ,((NTIMBF+5 + NTB)+1)):
                counter+=1
                index = int(i)
                data_dict_RA_greenwich['Right Ascension of Greenwich Values '+ str(counter)] = a[index-1] 


            ##### Satellite Data Packets
            #####    first satellite 
            #####    first time point 
            counter = 0        
            first_sat_first_time = ((NSATS +1)* NTIMBF +6) + (NSATS -1)* NWDSAT #2* NTIMBF +6
            last_sat_last_time   = ((NSATS +1)* NTIMBF +5) + NSATS*NWDSAT*NTB #(((NSATS+1)* NTIMBF+5)+(NSATS*NWDSAT))

            for i in np.arange(first_sat_first_time, last_sat_last_time  , 24):
                index = int(i)

                data_dict_sat_packets['MJDSEC ET'].append(data_dict_times[counter])
                data_dict_sat_packets['X j2000'].append(a[(index +1) - 2])
                data_dict_sat_packets['Y j2000'].append(a[(index +2) - 2])
                data_dict_sat_packets['Z j2000'].append(a[(index +3) - 2])
                data_dict_sat_packets['X_dot j2000'].append(a[(index +4) - 2])
                data_dict_sat_packets['Y_dot j2000'].append(a[(index +5) - 2])
                data_dict_sat_packets['Z_dot j2000'].append(a[(index +6) - 2])
                data_dict_sat_packets['Geodetic Latitude'].append(a[(index +7) - 2])
                data_dict_sat_packets['East Longitude'].append(a[(index +8) - 2])
                data_dict_sat_packets['Height'].append(a[(index +9) - 2])
    #                     data_dict_sat_packets['Satellite ECF X coordinate'].append(a[(index +10) - 2])
    #                     data_dict_sat_packets['Satellite ECF Y coordinate'].append(a[(index +11) - 2])
    #                     data_dict_sat_packets['Satellite ECF Z coordinate'].append(a[(index +12) - 2])
    #                     data_dict_sat_packets['Satellite ECF X velocity'].append(a[(index +13) - 2])
    #                     data_dict_sat_packets['Satellite ECF Y velocity'].append(a[(index +14) - 2])
    #                     data_dict_sat_packets['Satellite ECF Z velocity'].append(a[(index +15) - 2])
    #                     data_dict_sat_packets['Polar Motion X'].append(a[(index +16) - 2])
    #                     data_dict_sat_packets['Polar Motion Y'].append(a[(index +17) - 2])
    #                     data_dict_sat_packets['Beta prime angle'].append(a[(index +18) - 2])
    #                     data_dict_sat_packets['Yaw angle'].append(a[(index +19) - 2])
    #                     data_dict_sat_packets['Orbit Angle'].append(a[(index +20) - 2])
    #                     data_dict_sat_packets['Q(1)'].append(a[(index +21) - 2])
    #                     data_dict_sat_packets['Q(2)'].append(a[(index +22) - 2])
    #                     data_dict_sat_packets['Q(3)'].append(a[(index +23) - 2])
    #                     data_dict_sat_packets['Q(4)'].append(a[(index +24) - 2])
                counter+=1


    #         print('counter',counter)    

        else:
            ####  If the the first index has +9000000000 we are at the sentinel record 
            #     which denotes the end of the data section.
    #                 print('----- End of file')
    #                 print('sentinel buffer indicator                       ',a[1-1])
    #                 print('Count of the number of Data Buffers. GEODYN     ',a[2-1])
    #                 print('GEODYN II Interface File creation date and time.',a[3-1])
    #                 print('GEODYN II -S version used.                      ',a[4-1])
    #                 print('GEODYN II -E version used.                      ',a[5-1])
    #                 print('spare                                           ',a[6-1])
    #                 print('spare                                           ',a[7-1])
            end_datarecord = True
            f.close()  #### be sure to close the file


    data_record_df = pd.DataFrame.from_dict(data_dict_sat_packets, orient='columns')

    #### Save as a dictionary
    orbfil_dict = {}
    orbfil_dict['header'] = header
    orbfil_dict['data_record'] = data_record_df

    ##### Convert from Terrestrial time to UTC:
    MJDS_UTC = [time_mjdsecs_tdt_to_utc(float(x), 37) \
                for x in orbfil_dict['data_record']['MJDSEC ET'] ]

    ##### Calculate the Gregorian Calendar date:
    yymmdd_str = [mjds_to_ymdhms(x) for x in MJDS_UTC]

    ##### Compute date as Datetime object:
    dates_dt_UTC = [pd.to_datetime( x, format='%y%m%d-%H%M%S') for x in yymmdd_str]


    orbfil_dict['data_record']["Date_UTC"] = dates_dt_UTC
    orbfil_dict['data_record']["MJDS_UTC"] = MJDS_UTC

    return(orbfil_dict)

In [ ]:
gfo_traj_filename =  "/data/SatDragModelValidation/data/inputs/raw_inputdata/data_GRACEFO/traj_raw/geodyn.traj.20181101"

gfo_traj_filename =  "/data/SatDragModelValidation/data/inputs/raw_inputdata/data_GRACEFO/traj_raw/geodyn.traj.20181101"


# Read in the binary data into a dict w/ pd.dataframes
traj_data = read_binary_ORBFIL(gfo_traj_filename)



# sat_process_raw_ephemeris(verbose=True)

In [ ]:
traj_data['header']

In [ ]:
import sys
sys.exit(0)

In [ ]:
def get_timechopped_trajdata_1(path_binary, arc_files):
  
    
    total_files=np.size(arc_files)
    tabtab = '     '

    print(tabtab,'Running through the pre-processing procedure...')
    print(tabtab,'=======================================================')
    print(tabtab,'STEP 1: Convert TRAJ binary files to pandas DataFrame...')
    print(tabtab,'=======================================================')
    
    time_estimate_onAWS = 0.0005 # ~ num of secs per record ( 56.11/ 109434)
    rate = (total_files*time_estimate_onAWS*109000)/60
    print(tabtab, f"Loading and processing {total_files} files will take approx. {rate:.2f} minutes.")
    print(tabtab,tabtab, 'Not including unzipping/zipping times')

    print()
#         print(tabtab,'First we unzip the files... check the console for progess.')
    os.chdir(path_binary)


    # Initialize a Dataframe 
    df1 = pd.DataFrame()

    count=0
    for i, file in enumerate(arc_files):

        file = pd.to_datetime(file, format = '%Y.%j').strftime('%Y%m%d')
        
        filenum  =i+1 
        print(tabtab, '--- File %i / %i'% (filenum, total_files))

        print(tabtab, '----- Loading ', file  )


        __traj_filename =  path_binary + f"geodyn.traj.{file}"


        # Read in the binary data into a dict w/ pd.dataframes
        traj_data = read_binary_ORBFIL(__traj_filename)
        
        traj_data = pd.DataFrame.from_dict(traj_data['data_record']) 
            
        if count == 0:
            df1 = traj_data
            count += 1
        else:
            # to append df2 at the end of df1 dataframe
            df1 = pd.concat([df1, traj_data])


        df1 = df1.drop_duplicates(subset=["Date_UTC"], keep='first'\
        ).sort_values(by='Date_UTC'\
                        ).reset_index(drop=True)

        del traj_data
        gc_collect()

#         print(tabtab,'Zipping file...', file)
#             os.system('gzip -v '+file)

        print()

    TRAJ_FINAL = df1
    return(TRAJ_FINAL)





In [ ]:
def sat_process_raw_ephemeris( verbose=False):


    from datetime import datetime, timedelta
    import time
    
    raw_satinput = {}
    prms = {}
    path_data_inputs = '/data/SatDragModelValidation/data/inputs'
    raw_satinput['ephem_path']  = path_data_inputs +'/'\
                            +f'sat_gracefoc/g2b/'  \
                            +f'GRACEFOC_RawEphem_test_v1.txt'

    raw_satinput['ephem_path_dir'] = '/data/SatDragModelValidation/data/inputs/raw_inputdata/data_GRACEFO/traj_raw/'
    prms['epoch_start'] = ['2018-11-01 00:00:00']
    prms['arc'] = ['2018.305']

    tabtab= '           '

    prms['sat_ID'] = 2012003
    
    

    raw_ephem = raw_satinput['ephem_path']
    ephem_path_dir = raw_satinput['ephem_path_dir']
    #
    dt_2days = pd.Series(pd.to_timedelta(48,'h'))
    startdate = pd.to_datetime(prms['epoch_start'][0])
    enddate   = pd.to_datetime(prms['epoch_start'][-1])
    startdate_dt = pd.to_datetime(startdate, format='%Y-%m-%d')
    enddate_dt   = pd.to_datetime(enddate,   format='%Y-%m-%d')
    starts_linspace_dt = pd.date_range(start=startdate_dt ,
                                        end=enddate_dt   ,
                                        freq=str(1)+"D")

    if verbose: print(f"{tabtab} - GRACEFOC")
    if verbose: print(f"{tabtab} - processing raw satellite ephemerides from files.")
    if verbose: print(f"{tabtab} - for dates: {starts_linspace_dt}")


    ### PLACE THE RAW FILE PROCESSING CODE HERE
    ### ---------------------------------------

    ORBFIL_FINAL = get_timechopped_trajdata_1(ephem_path_dir, \
                                        prms['arc'],\
                                        )
    ### ---------------------------------------



    start = time.time()

    ### Date must be in UTC
    ### Ephemerides must be in ECI-J2000

    if verbose: print(f"{tabtab}   saving satellite ephemeris to single file.")
    f = open(raw_ephem, "w")
    f.write("\n")
    f.close()


    ### Write to file
    with open(raw_ephem, 'r+') as file:

        #### Manually write the header units
        header_units =\
                f"{'UTC'.rjust(19-1,' ') }"\
                +f"  {'(m)'.rjust(15,' ')}"\
                +f"  {'(m)'.rjust(15,' ')}"\
                +f"  {'(m)'.rjust(15,' ')}"\
                +f"  {'(m/s)'.rjust(15,' ')}"\
                +f"  {'(m/s)'.rjust(15,' ')}"\
                +f"  {'(m/s)'.rjust(15,' ')}"\

        #### Manually write the header field names
        header_names =\
                f"{'Date'.rjust(19-1,' ') }"\
                +f"  {'X'.rjust(15,' ')}"\
                +f"  {'Y'.rjust(15,' ')}"\
                +f"  {'Z'.rjust(15,' ')}"\
                +f"  {'X_dot'.rjust(15,' ')}"\
                +f"  {'Y_dot'.rjust(15,' ')}"\
                +f"  {'Z_dot'.rjust(15,' ')}"\

        #### Manually write the detailed header description
        header_meta = \
        f'''### "Raw" Satellite Ephemeris
### -----------------------
###     Satellite: GRACE-FO-C_({prms['sat_ID']})
###     Epoch: +start____ {ORBFIL_FINAL['Date_UTC'].values[0]} 
###            +stop_____ {ORBFIL_FINAL['Date_UTC'].values[-1]}
###     Last modified: {datetime.now()-timedelta(hours=7)}
###
### Source
### -------
###     {raw_satinput['ephem_path_dir']}
###     GRACE-FO-C PSO, Orbit Trajectory binary files
###
### Contents
### --------
###     Date:  (YYYY-MM-DD hh:mm:ss) (UTC, converted from mjdsec-gps time)
###     Ephem:  Position and velocity (X, Y, Z, X_dot, Y_dot, Z_dot)
###             coordinate: ECI-J2000
###             unit: m
###
###
#{header_units}
#{header_names}
### %eoh
'''
        print(np.shape(ORBFIL_FINAL['X j2000'].values))

        
        
        file.write(header_meta)  
        for indx,valdate in enumerate(ORBFIL_FINAL['Date_UTC'].values):
        #### Manually write each row of the data.
            row =   f"{pd.to_datetime(ORBFIL_FINAL['Date_UTC'].values[indx]).strftime(format='%Y-%m-%d %H:%M:%S')}"\
                +f"  {ORBFIL_FINAL['X j2000'].values[indx]:15.5f}"\
                +f"  {ORBFIL_FINAL['Y j2000'].values[indx]:15.5f}"\
                +f"  {ORBFIL_FINAL['Z j2000'].values[indx]:15.5f}"\
                +f"  {ORBFIL_FINAL['X_dot j2000'].values[indx]:15.5f}"\
                +f"  {ORBFIL_FINAL['Y_dot j2000'].values[indx]:15.5f}"\
                +f"  {ORBFIL_FINAL['Z_dot j2000'].values[indx]:15.5f}"\
                +f"\n"
            file.write(row)
    #
    end = time.time()
    elapsed = end - start
    #
    print()
    # print(f'       indxes: 0 -',indx,'') 
    print(f'       Processed file in : ',np.round(elapsed,5),'secs', f"({np.round(elapsed,5)/60} minutes)") 



# Read ExAtt Files

In [ ]:
def read_EXTAT_binary():
    """
        OVERVIEW  
            External Attitude File is a binary file where all records contain 9, 64-bit real words
            +-----------------------------------------------------------
            |Nomenclature
            |    SBF   --> Spacecraft  Body  Fixed  Frame
            |    J2000 --> J2000  Earth  Equator  and  Equinox  Frame
            |    MVP   -->  Movable  Panel  Frame
            |    MVA   -->  Movable  Antenna  Frame
            |        ("Movable" is with  respect  to the SBF  frame)
            |        (All  times in this  file  are TDT or TDB)
            |              TDT (Terrestrial  Dynamic  Time)
            |              TDB (Barycenter   Dynamic  Time)
            |              GPS (Global  Positioning  System Time)
            +------------------------------------------------------------

        The file is segmented as follows:
            1) GENERAL HEADER RECORD
            2) SATELLITE INFORMATION HEADER RECORDS
            3) QUATERNION  SET  HEADER  RECORD
            4) DATA RECORDS



                1)  GENERAL HEADER RECORD
                    --------------------- 
                    There is only one of these records and it must be first in the file.
                -----------------------------------------------------------------------
                WORD         DESCRIPTION
                ----------------------------------------------------------------------- 
                1          Record Indicator                              -6666666.00
                2          Version Number
                3          Number of Satellites represented in this file.
                4          Not used at present time
                5          Not used at present time
                6          Not used at present time
                7          Not used at present time
                8          Not used at present time
                9          Not used at present time


                2)  SATELLITE INFORMATION HEADER RECORDS
                    ------------------------------------ 
                    The number of these records equals the number of satellites from the 
                    General Header Record. All of these records must directly follow the
                    above General Header Record.
                -----------------------------------------------------------------------
                WORD         DESCRIPTION
                ----------------------------------------------------------------------- 
                1           Record Indicator                      -7777777.00
                2           Not used at present time              
                3           Satellite ID*                         
                4           Interval                              SSSSS.SSSSS
                5           Start time                            YYMMDDHHMMSS .00
                6           Start (fractional seconds)            00.SS
                7           Stop time                             YYMMDDHHMMSS .00
                8           Stop (fractional seconds)             00.SS
                9           No. of: panels+antenna separate       QQQPPPAAA **
                                quaternion sets/movable panels
                                represented/movable antenna 
                                represented        

                3)  QUATERNION SET HEADER RECORD
                    ----------------------------
                    This header record must precede the quaternion data 
                    records for a particular set of quaternions.
                -----------------------------------------------------------------------
                WORD           DESCRIPTION
                -----------------------------------------------------------------------
                1           Record  Indicator                    -8888888.00
                2           Satellite  ID
                3           Panel  Number*                        MMMNNN
                4           Antenna -Link  Number ***
                5           Start  time**                         YYMMDDHHMMSS .00
                6           Start (fractional  seconds )**        00.SS
                7           Stop  time**                          YYMMDDHHMMSS .00
                8           Stop (fractional  seconds )**         00.SS
                9           Interval **                           SSSSS.SSSSS

    """
    from scipy.io import FortranFile


    
    AttitudeFile = '/data/SatDragModelValidation/data/inputs/sat_gracefoc/external_attitude/EXAT01_AB_2018303115947_2018305115947'
    f = FortranFile(AttitudeFile, 'r')

    sp = '    '
    header= {}

    ### ----------------------------------------------------
    ### ------------- 1) GENERAL HEADER RECORD -------------
    ###
    ###      Read the first record, this is the header buffer
    ####     Use 64-bit float datatype. Each record contains 9, 64-bit words
    a = f.read_record(float)  

    if a[1 -1] == -6666666.00:
        print('Reading GENERAL HEADER RECORD')

        header['NSATS']           = a[3 -1]  # Number of Satellites represented in this file.
        print(sp, 'NSATS =', header['NSATS'] )
    for sats in np.arange(1, int(header['NSATS'])+1):
        header['SATINFO_'+f"{int(sats):02}"] = {}
    #
    #
    ### ----------------------------------------------------
    ### ---------- 2) SATELLITE INFORMATION RECORD ---------
    ###  
    a = f.read_record(float)
    if a[1 -1] == -7777777.00:
        print('Reading SATELLITE INFORMATION HEADER RECORD')

        for i in np.arange(1, int(header['NSATS'])+1):
            iSat = 'SATINFO_'+f"{int(i):02}"

            header[iSat]['SATID']                             = a[3 -1]  

            header[iSat]['Interval (SSSSS.SSSSS)']            = a[4 -1]  
            header[iSat]['Start time (YYMMDDHHMMSS.00)']      = a[5 -1]  
            header[iSat]['Start (fractional seconds, 00.SS)'] = a[6 -1] 
            header[iSat]['Stop time (YYMMDDHHMMSS.00)']       = a[7 -1] 
            header[iSat]['Stop (fractional seconds, 00.SS)']  = a[8 -1]  
            header[iSat]['QQQPPPAAA']                         = a[9 -1]  

            datestart_string = str(header[iSat]['Start time (YYMMDDHHMMSS.00)']).split('.')[0]
            datestop_string  = str(header[iSat]['Stop time (YYMMDDHHMMSS.00)']).split('.')[0]
            date_start       = pd.to_datetime(datestart_string, format='%y%m%d%H%M%S')
            date_stop        = pd.to_datetime(datestop_string, format='%y%m%d%H%M%S')

            print(sp,'Start:',date_start)
            print(sp,'Stop:' ,date_stop)
            print(sp,'Interval:',header[iSat]['Interval (SSSSS.SSSSS)'])

            print(sp,'QQQPPPAAA', a[9 -1])
                # QQQ is the total # of separate movable panels + antenna quaternion  
                #     sets for this satellite (excludes SBF to J2000 quaternion set  
                #     which is mandatory for each satellite represented in the file).   
                # PPP is the number of movable panels represented for this satellite. 
                # AAA is the number of moveable antenna represented for this  satellite.
                #     One quaternion set may represent the attitude for up to two  
                #     movable panels and one movable antenna.                
            print('SatID',header[iSat]['SATID'])


    #
    #
    ### -------------------------------------------------------
    ### ---------- 3) QUATERNION SET HEADER RECORD  --------
    ###  
    ###       This header record must precede the 
    ###            quaternion data records for a 
    ###            particular set of quaternions.


    def read_QUAT_SET_HEADER(set_count, header, a):
        if a[1 -1] ==  -8888888.00:
    #         print(a)
            set_count+=1
            iQuat = "QuatSet"+str(set_count)
            print('Reading QUATERNION SET HEADER RECORD', str(set_count))

            header[iQuat] = {}

            header[iQuat]['SATID']                            = a[2 -1] 
            print('SatID',header[iQuat]['SATID'])
            header[iQuat]['Panel # (MMMNNN)']                 = a[3 -1] 
                #    MMM is 1st panel  number  and 
                #    NNN is 2nd panel  number
            header[iQuat]['AntennaLink #']                    = a[4 -1]  
                #Antenna -Link  number  is used to  specify  both  the  antenna  and  linknumber(see  table  below ).
            ####
            ###  Qaternion  satellite  information  must  match  
            ###  that on thes satellite  information  header  record.
            header[iQuat]['Start time (YYMMDDHHMMSS.00)']      = a[5 -1]  
            header[iQuat]['Start (fractional seconds, 00.SS)'] = a[6 -1] 
            header[iQuat]['Stop time (YYMMDDHHMMSS.00)']       = a[7 -1] 
            header[iQuat]['Stop (fractional seconds, 00.SS)']  = a[8 -1]  
            header[iQuat]['Interval (SSSSS.SSSSS)']            = a[9 -1]  

        else:
            print(a)
            f.close()  #### be sure to close the file
            sys.exit(0)
        return(set_count, header)


    #### Read the first header record
    a = f.read_record(float)
    set_count = 0
    (set_count, header) = read_QUAT_SET_HEADER(set_count, header, a)



    def read_QUAT_DATA(a, data, set_count, header):
        print("Reading Data for set:", set_count)
        data[set_count] = {}
        data[set_count]['q1'] = []
        data[set_count]['q2'] = []
        data[set_count]['q3'] = []
        data[set_count]['q4'] = []

        while a[1 -1] ==  0:
            data[set_count]['q1'].append(a[3 -1])  ###  sin (/2)n1
            data[set_count]['q2'].append(a[4 -1])  ###  sin (/2)n2
            data[set_count]['q3'].append(a[5 -1])  ###  sin (/2)n3
            data[set_count]['q4'].append(a[6 -1])  ###  cos (/2)

            ### Move to next data record while in the loop
            a = f.read_record(float)
        else:
            if a[1 -1] !=  0:
    #             print('Back to a header?')
                (set_count, header) = read_QUAT_SET_HEADER(set_count, header, a)
                try:
                    a = f.read_record(float)
                    (a, data, set_count) = read_QUAT_DATA(a, data, set_count, header)

                except:
                    print('End of File')
                    f.close()  #### be sure to close the file

        return(a, data, set_count)



    ### Read the first Data record and then open the loop until the end of file
    a = f.read_record(float)
    data = {}
    (a, data, set_count) = read_QUAT_DATA(a, data, set_count, header)
    f.close()  #### be sure to close the file

    ## attempt to clear out some memory
    f         = 0
    a         = 0
    set_count = 0

    return()



read_EXTAT_binary()